# Binary puzzle/ Binario/ Takuzu puzzle
## Bruteforce puzzle generator (xD)
https://en.wikipedia.org/wiki/Takuzu

Rules:
1. Each row or column should have equal number of 1s and 0s
1. Cannot have more than 2 grouped entries horizontally or vertically

Approach:
1) Is checked via sum on proper axis
2) Is checked via convolution/cross-correlation

NOTE: Cannot easily batch convolutions using scipy. <br> 
NOTE: For fast approach see same implementation for GPU using Pytorch: [Binario_puzzle_generator_GPU.ipynb](../../multiprocessing/GPU/Binario_puzzle_generator_GPU.ipynb) (Bit better quality. very fast)

In [18]:
from scipy import signal
import numpy as np

three_horiz = np.array([[0,0,0],
                        [1,1,1],
                        [0,0,0]])

three_vert  = np.array([[0,1,0],
                        [0,1,0],
                        [0,1,0]])

nb_cnt_3= lambda puzz, ker: (signal.convolve2d(puzz, ker, boundary='fill', mode='same', fillvalue=0) == 3 ).astype(int)

def puzzle_ok_CPU(puzz):
    D = puzz.shape[0]//2
    # check if any entries have 2 neighbors (+1 self)
    neighb_failed  = (  nb_cnt_3(  puzz, three_horiz) |
                        nb_cnt_3(  puzz, three_vert ) |
                        nb_cnt_3(1-puzz, three_horiz) | # invert 0s and 1s (for sum to work)
                        nb_cnt_3(1-puzz, three_vert ) ) # invert 0s and 1s
    
    three_failed = np.any(neighb_failed)
    # check element count. 
    # in case of even N num 1s = num 0s. 
    # we dont count 0s
    vert_num_elems_failed   = np.any(np.sum(puzz, axis = 0) != D)
    horiz_num_elems_failed  = np.any(np.sum(puzz, axis = 1) != D)
    num_failed = vert_num_elems_failed or horiz_num_elems_failed

    return ~(three_failed or num_failed)

puzz = np.array([[1,0,1,0,1,0],
                 [0,1,0,0,1,1],
                 [1,0,0,1,0,1],
                 [0,1,1,0,1,0],
                 [0,0,1,1,0,1],
                 [1,1,0,1,0,0]])

print(f'Puzzle solved properly: {puzzle_ok_CPU(puzz)}')


True

In [19]:
from tqdm import tqdm
N = 6
B = 10_000_000
puzzles = np.uint8(np.random.rand(B,N,N)>0.5)
for i in tqdm(range(B)):
    np.random.seed(i)
    puzz = puzzles[i]
    if puzzle_ok_CPU(puzz):
        print(puzz)
        print(i)
        break

 70%|███████   | 7041061/10000000 [10:12<04:17, 11490.10it/s]

[[0 1 1 0 1 0]
 [0 1 0 1 0 1]
 [1 0 1 1 0 0]
 [0 1 0 0 1 1]
 [1 0 0 1 1 0]
 [1 0 1 0 0 1]]
7041061
